# Multi i-TED: CRT study
## Fourth: More combinations and position cuts

- Study of CRT using different algorithms:
    - First event (SKEW=0)
    - Average of the first n events (DEGREE=0)
    - Energy-weighted average of the first n events (DEGREE=1)
    - Second power of the enery-weighted average of the first n events (DEGREE=2)
    - Third power of the enery-weighted average of the first n events (DEGREE=3)
- The measurement using Na22 for timing studies:
    - It includes filters around the 511keV energy peak applied per crystal
    - Coincidences between the scatterer and all of the absorbers are taken into account
    - Gaussian fits are used to determine the standard deviation
- The calculation of the timing:
    - The timing weighted is calculated as energy^DEGREE
    - In case energy^DEGREE causes an error (`floating point exception`), then it is considered 0 and not taking into account for the average
    - In case the sum of the energy^DEGREE is 0, then the time will be 0, as all weights were too low

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

import copy

uproot              4.3.5
Welcome to JupyROOT 6.28/02
sklearn             1.2.2
torch               2.0.0
scipy               1.10.1
numpy               1.23.5
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
file = lambda skew, degree: f"/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTEDD_timing_D.2023_04_18_T.11_31_52_C.itedABCD_lab_2023.02.22_4.0v_888_120s_CW100_{skew}_{degree}.root"

In [4]:
skew = [i+1 for i in range(25)]
degree = [0,1,2,3]

spectra = pandas.DataFrame(index = skew, columns = degree)

In [5]:
for i_skew in tqdm.tqdm(skew):
    for i_degree in degree:
        f_root = ROOT.TFile.Open(file(i_skew,i_degree))
        t_root = f_root.COINCIDENCES
        
        canvas = ROOT.TCanvas()
        canvas.cd()
        
        t_root.Draw("Delta_t[15]-Delta_t[16]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        h_root = ROOT.gPad.GetPrimitive("htemp")
                
        #t_root.Draw("Delta_t[15]-Delta_t[17]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        #h_root1 = ROOT.gPad.GetPrimitive("htemp")
        #h_root.Add(h_root1)
        
        #t_root.Draw("Delta_t[15]-Delta_t[18]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        #h_root2 = ROOT.gPad.GetPrimitive("htemp")
        #h_root.Add(h_root2)
        
        #t_root.Draw("Delta_t[15]-Delta_t[19]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        #h_root3 = ROOT.gPad.GetPrimitive("htemp")
        #h_root.Add(h_root2)
        
        h_root.Draw()
                
        latex = ROOT.TLatex()
        latex.SetNDC()
        latex.SetTextSize(0.03)
        latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
        latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        
        
        gaussFit = ROOT.TF1("gaussFit", "gaus", -5.0, 5.0)
        h_root.Fit(gaussFit,"QR")

        sigma = abs(gaussFit.GetParameter(2))
        centroid_ch = gaussFit.GetParameter(1)
        
        spectra[i_degree][i_skew] = {
            "histo":canvas,
            "mean":h_root.GetMean(),
            "mean_fit":centroid_ch,
            "std":h_root.GetStdDev(),
            "std_fit":sigma,
            "skew":h_root.GetSkewness(),
            "kurt":h_root.GetKurtosis(),
        }

100%|████████████████| 25/25 [03:03<00:00,  7.33s/it]


## Analysis

### Standard deviation

In [6]:
spectra.applymap(lambda x: x["std_fit"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,1.461833,1.461833,1.461833,1.461833
2,1.353935,1.446548,1.523530,1.496958
3,1.285825,1.433603,1.481470,1.487713
4,1.231788,1.417217,1.472548,1.492461
5,1.166858,1.398856,1.462067,1.504304
6,1.110786,1.376599,1.459354,1.543133
7,1.067434,1.342077,1.464800,1.581459
8,1.033222,1.318095,1.475755,1.626040
9,1.017052,1.285803,1.490974,1.667020
10,1.017445,1.251463,1.500214,1.721150


In [7]:
spectra.applymap(lambda x: x["std"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,1.723254,1.723254,1.723254,1.723254
2,1.850902,2.220804,1.960248,1.779143
3,2.003701,2.215663,1.874325,1.740803
4,2.105275,2.127280,1.794819,1.713594
5,2.172813,2.009810,1.745051,1.686730
6,2.253340,1.922061,1.708771,1.703135
7,2.362952,1.840037,1.695277,1.724941
8,2.388269,1.751664,1.685457,1.760897
9,2.321521,1.681038,1.685813,1.795132
10,2.208307,1.610165,1.685929,1.820711


### Mean value, skewness, and kurtosis

In [8]:
spectra.applymap(lambda x: x["mean_fit"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,-0.650543,-0.650543,-0.650543,-0.650543
2,-0.719473,-0.765986,-0.797317,-0.777255
3,-0.755937,-0.828858,-0.845725,-0.842560
4,-0.783197,-0.872681,-0.894242,-0.903176
5,-0.783704,-0.896462,-0.934571,-0.958438
6,-0.770959,-0.908688,-0.966464,-1.011811
7,-0.753980,-0.913734,-0.997270,-1.058682
8,-0.729392,-0.915269,-1.023513,-1.103910
9,-0.695171,-0.911475,-1.046051,-1.149260
10,-0.655495,-0.898742,-1.068238,-1.194389


In [9]:
spectra.applymap(lambda x: x["skew"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,0.108488,0.108488,0.108488,0.108488
2,0.117829,0.078990,0.302431,0.372200
3,0.060835,0.114138,0.373605,0.276017
4,0.201416,0.264044,0.386125,0.426785
5,0.139670,0.343924,0.424563,0.385765
6,0.200081,0.326988,0.476954,0.439212
7,0.145276,0.311058,0.412891,0.426294
8,0.147138,0.317352,0.363078,0.429305
9,0.097788,0.333541,0.340775,0.386171
10,0.066025,0.319722,0.346376,0.385368


In [10]:
spectra.applymap(lambda x: x["kurt"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,4.552298,4.552298,4.552298,4.552298
2,6.809468,5.152239,4.661985,4.367410
3,7.059212,4.676600,4.715399,4.171538
4,6.852992,4.756846,4.348892,3.706981
5,6.654678,4.956530,4.140099,2.805881
6,6.534908,5.065603,3.769525,2.640814
7,6.076792,5.222734,3.622329,2.425688
8,5.706663,5.181162,3.265025,2.223504
9,5.427044,5.195241,3.110456,2.040270
10,5.218802,5.061761,2.886934,1.701601


## Absorber and scatterer cuts

- Energy weighted
    - The position in the absorber doesn't seem to influence nearly as much the resolution as the position in the scatterer
    - A cut of the possible possitions seems to yield a better result (about .5%)
    - Further restricting the scatterer in 8mm yields an improvement of 3.5% in relation to the unrestricted case
- Normal average
    - Position cuts in the absorber also improve the resolution
    - The best result with a resonable cut is worse (+.78%) than the energy-weighted method, while requiring a bigger cut and, as such, less events

In [11]:
cuts = [i for i in range(-5,11)]

spectra_cuts = pandas.DataFrame(index = cuts, columns = cuts)
spectra_cuts1 = pandas.DataFrame(index = cuts, columns = cuts)

spectra_cuts.index.rename("Cut Scatterer",inplace=True)
spectra_cuts1.index.rename("Cut Scatterer",inplace=True)

In [12]:
for cut_s in tqdm.tqdm(cuts):
    for cut in cuts:
        for i_skew, i_degree in [[25,1]]:
    
            f_root = ROOT.TFile.Open(file(i_skew,i_degree))
            t_root = f_root.COINCIDENCES

            canvas = ROOT.TCanvas()
            canvas.cd()

            t_root.Draw(
                "Delta_t[15]-Delta_t[16]",
               f"deposited_energy[15]>160.0&deposited_energy[15]<195.0&\
                &deposited_energy[16]>125.0&deposited_energy[16]<165.0&\
                &x[16]>-{25-cut}&x[16]<{25-cut}&\
                &y[16]>-{25-cut}&y[16]<{25-cut}&\
                &x[15]>-{25-cut_s}&x[15]<{25-cut_s}&\
                &y[15]>-{25-cut_s}&y[15]<{25-cut_s}",
                ""
            )
            h_root = ROOT.gPad.GetPrimitive("htemp")

            #t_root.Draw("Delta_t[15]-Delta_t[17]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
            #h_root1 = ROOT.gPad.GetPrimitive("htemp")
            #h_root.Add(h_root1)

            #t_root.Draw("Delta_t[15]-Delta_t[18]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
            #h_root2 = ROOT.gPad.GetPrimitive("htemp")
            #h_root.Add(h_root2)

            #t_root.Draw("Delta_t[15]-Delta_t[19]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
            #h_root3 = ROOT.gPad.GetPrimitive("htemp")
            #h_root.Add(h_root2)

            h_root.Draw()

            latex = ROOT.TLatex()
            latex.SetNDC()
            latex.SetTextSize(0.03)
            latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
            latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        

            gaussFit = ROOT.TF1("gaussFit", "gaus", -5.0, 5.0)
            h_root.Fit(gaussFit,"QR")

            sigma = abs(gaussFit.GetParameter(2))
            centroid_ch = gaussFit.GetParameter(1)

            spectra_cuts[cut][cut_s] = {
                "histo":canvas,
                "mean":h_root.GetMean(),
                "mean_fit":centroid_ch,
                "std":h_root.GetStdDev(),
                "std_fit":sigma,
                "skew":h_root.GetSkewness(),
                "kurt":h_root.GetKurtosis(),
            }

100%|████████████████| 16/16 [02:12<00:00,  8.30s/it]


In [13]:
spectra_cuts.applymap(lambda x: x["std_fit"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10
Cut Scatterer,,,,,,,,,,,,,,,,
-5,1.002812,1.002773,1.002440,1.001500,1.000979,1.000161,1.000762,1.001159,1.000854,1.001969,1.002519,1.003078,1.002130,1.001112,1.002002,1.008094
-4,1.002684,1.002646,1.002309,1.001370,1.000852,1.000036,1.000635,1.001029,1.000728,1.001845,1.002389,1.002942,1.001992,1.000972,1.001865,1.007948
-3,1.002290,1.002252,1.001912,1.000973,1.000449,0.999624,1.000215,1.000605,1.000288,1.001406,1.001955,1.002487,1.001525,1.000485,1.001369,1.007380
-2,1.001251,1.001211,1.000871,0.999917,0.999383,0.998543,0.999121,0.999495,0.999252,1.000366,1.000904,1.001398,1.000390,0.999239,0.999970,1.006157
-1,1.001313,1.001275,1.000927,0.999974,0.999437,0.998584,0.999118,0.999486,0.999228,1.000338,1.000883,1.001329,1.000287,0.999106,0.999835,1.005917
0,1.000207,1.000167,0.999825,0.998855,0.998300,0.997436,0.997956,0.998311,0.998124,0.999157,0.999668,1.000075,0.999070,0.997833,0.998295,1.004262
1,0.996723,0.996711,0.996385,0.995406,0.994844,0.994012,0.994503,0.994697,0.994484,0.995270,0.995699,0.996503,0.995884,0.994321,0.995135,1.001094
2,0.995760,0.995748,0.995417,0.994413,0.993828,0.992971,0.993423,0.993706,0.993429,0.994300,0.994674,0.995396,0.994814,0.993099,0.993924,0.999716
3,0.994492,0.994481,0.994145,0.993100,0.992514,0.991707,0.992148,0.992534,0.992306,0.993155,0.993483,0.994423,0.994299,0.992454,0.993047,0.998930


In [14]:
for cut_s in tqdm.tqdm(cuts):
    for cut in cuts:
        for i_skew, i_degree in [[9,0]]:
    
            f_root = ROOT.TFile.Open(file(i_skew,i_degree))
            t_root = f_root.COINCIDENCES

            canvas = ROOT.TCanvas()
            canvas.cd()

            t_root.Draw(
                "Delta_t[15]-Delta_t[16]",
               f"deposited_energy[15]>160.0&deposited_energy[15]<195.0&\
                &deposited_energy[16]>125.0&deposited_energy[16]<165.0&\
                &x[16]>-{25-cut}&x[16]<{25-cut}&\
                &y[16]>-{25-cut}&y[16]<{25-cut}&\
                &x[15]>-{25-cut_s}&x[15]<{25-cut_s}&\
                &y[15]>-{25-cut_s}&y[15]<{25-cut_s}",
                ""
            )
            h_root = ROOT.gPad.GetPrimitive("htemp")

            #t_root.Draw("Delta_t[15]-Delta_t[17]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
            #h_root1 = ROOT.gPad.GetPrimitive("htemp")
            #h_root.Add(h_root1)

            #t_root.Draw("Delta_t[15]-Delta_t[18]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
            #h_root2 = ROOT.gPad.GetPrimitive("htemp")
            #h_root.Add(h_root2)

            #t_root.Draw("Delta_t[15]-Delta_t[19]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
            #h_root3 = ROOT.gPad.GetPrimitive("htemp")
            #h_root.Add(h_root2)

            h_root.Draw()

            latex = ROOT.TLatex()
            latex.SetNDC()
            latex.SetTextSize(0.03)
            latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
            latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        

            gaussFit = ROOT.TF1("gaussFit", "gaus", -5.0, 5.0)
            h_root.Fit(gaussFit,"QR")

            sigma = abs(gaussFit.GetParameter(2))
            centroid_ch = gaussFit.GetParameter(1)

            spectra_cuts1[cut][cut_s] = {
                "histo":canvas,
                "mean":h_root.GetMean(),
                "mean_fit":centroid_ch,
                "std":h_root.GetStdDev(),
                "std_fit":sigma,
                "skew":h_root.GetSkewness(),
                "kurt":h_root.GetKurtosis(),
            }

spectra_cuts1.applymap(lambda x: x["std_fit"]).style.background_gradient(cmap ='YlOrRd',axis=None)

100%|████████████████| 16/16 [02:07<00:00,  7.95s/it]


,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10
Cut Scatterer,,,,,,,,,,,,,,,,
-5,1.017092,1.017020,1.016309,1.015408,1.014793,1.015170,1.015949,1.016363,1.016257,1.015484,1.014472,1.012751,1.011266,1.009715,1.009714,1.016677
-4,1.017092,1.017020,1.016309,1.015408,1.014793,1.015170,1.015949,1.016363,1.016257,1.015484,1.014472,1.012751,1.011266,1.009715,1.009714,1.016677
-3,1.016934,1.016862,1.016152,1.015253,1.014637,1.015013,1.015793,1.016205,1.016096,1.015320,1.014314,1.012585,1.011085,1.009529,1.009501,1.016451
-2,1.014284,1.014214,1.013510,1.012618,1.011989,1.012356,1.013107,1.013461,1.013570,1.012753,1.011683,1.009934,1.008295,1.006644,1.006703,1.013278
-1,1.011271,1.011204,1.010500,1.009617,1.008973,1.009325,1.010044,1.010332,1.010380,1.009489,1.008297,1.006475,1.004585,1.002728,1.002429,1.008649
0,1.008897,1.008830,1.008126,1.007239,1.006586,1.007038,1.007741,1.008013,1.008267,1.007460,1.006199,1.004219,1.002330,1.000264,1.000204,1.006045
1,1.002800,1.002734,1.002034,1.001163,1.000488,1.000923,1.001563,1.001638,1.001870,1.000903,0.999558,0.997571,0.995970,0.993730,0.993610,0.999404
2,0.997115,0.997050,0.996333,0.995451,0.994740,0.995142,0.995743,0.995867,0.995921,0.994987,0.993423,0.991054,0.989069,0.986517,0.986710,0.992178
3,0.993338,0.993276,0.992548,0.991671,0.990889,0.991234,0.991793,0.992125,0.992289,0.991180,0.989687,0.987265,0.985763,0.982835,0.982986,0.988501


## Position dependency

- Many quadrants don't have enough statistics
- The ones that do show consistent values of resolution

In [15]:
cells = [i for i in range(-25,25,5)]

midx_s = pandas.MultiIndex.from_product(
    [cells],
    names=['Scatterer_x']
)

mcol_s = pandas.MultiIndex.from_product(
    [cells],
    names=['Scatterer_y']
)

midx_a = pandas.MultiIndex.from_product(
    [cells],
    names=['Absorber_x']
)

mcol_a = pandas.MultiIndex.from_product(
    [cells],
    names=['Absorber_y']
)

spectra_cells_s = pandas.DataFrame(index = midx_s, columns = mcol_s)

spectra_cells_a = pandas.DataFrame(index = midx_a, columns = mcol_a)

In [16]:
for s_x in tqdm.tqdm(spectra_cells_s.index):
    for s_y in spectra_cells_s.columns:
        for i_skew, i_degree in [[25,1]]:
            
            f_root = ROOT.TFile.Open(file(i_skew,i_degree))
            t_root = f_root.COINCIDENCES
            
            canvas = ROOT.TCanvas()
            canvas.cd()
                        
            t_root.Draw(
                "Delta_t[15]-Delta_t[16]",
               f"deposited_energy[15]>160.0&deposited_energy[15]<195.0&\
                &deposited_energy[16]>125.0&deposited_energy[16]<165.0&\
                &x[16]>-25&x[16]<25&\
                &y[16]>-25&y[16]<25&\
                &x[15]>{s_x[0]}&x[15]<{s_x[0]+5}&\
                &y[15]>{s_y[0]}&y[15]<{s_y[0]+5}",
                ""
            )

            h_root = ROOT.gPad.GetPrimitive("htemp")
            
            if "cppyy.gbl.TObject" in str(type(h_root)):
                continue

            h_root.Draw()

            latex = ROOT.TLatex()
            latex.SetNDC()
            latex.SetTextSize(0.03)
            latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
            latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        

            gaussFit = ROOT.TF1("gaussFit", "gaus", -6.0, 5.0)
            h_root.Fit(gaussFit,"QR")

            sigma = abs(gaussFit.GetParameter(2))
            centroid_ch = gaussFit.GetParameter(1)

            spectra_cells_s[s_y][s_x] = {
                "histo":canvas,
                "count":h_root.Integral(),
                "mean":h_root.GetMean(),
                "mean_fit":centroid_ch,
                "std":h_root.GetStdDev(),
                "std_fit":sigma,
                "skew":h_root.GetSkewness(),
                "kurt":h_root.GetKurtosis(),
                "cut":f"SX{s_x}\nSY{s_y}"
            }

for a_x in tqdm.tqdm(spectra_cells_a.index):
    for a_y in spectra_cells_a.columns:
        for i_skew, i_degree in [[25,1]]:
            
            f_root = ROOT.TFile.Open(file(i_skew,i_degree))
            t_root = f_root.COINCIDENCES
            
            canvas = ROOT.TCanvas()
            canvas.cd()
            
            t_root.Draw(
                "Delta_t[15]-Delta_t[16]",
               f"deposited_energy[15]>160.0&deposited_energy[15]<195.0&\
                &deposited_energy[16]>125.0&deposited_energy[16]<165.0&\
                &x[16]>{a_x[0]}&x[16]<{a_x[0]+5}&\
                &y[16]>{a_y[0]}&y[16]<{a_y[0]+5}&\
                &x[15]>-25&x[15]<25&\
                &y[15]>-25&y[15]<25",
                ""
            )

            h_root = ROOT.gPad.GetPrimitive("htemp")
            
            if "cppyy.gbl.TObject" in str(type(h_root)):
                continue

            h_root.Draw()

            latex = ROOT.TLatex()
            latex.SetNDC()
            latex.SetTextSize(0.03)
            latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
            latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        

            gaussFit = ROOT.TF1("gaussFit", "gaus", -6.0, 5.0)
            h_root.Fit(gaussFit,"QR")

            sigma = abs(gaussFit.GetParameter(2))
            centroid_ch = gaussFit.GetParameter(1)

            spectra_cells_a[a_y][a_x] = {
                "histo":canvas,
                "count":h_root.Integral(),
                "mean":h_root.GetMean(),
                "mean_fit":centroid_ch,
                "std":h_root.GetStdDev(),
                "std_fit":sigma,
                "skew":h_root.GetSkewness(),
                "kurt":h_root.GetKurtosis(),
                "cut":f"AX{a_x}\nAY{a_y}"
            }

100%|████████████████| 10/10 [00:45<00:00,  4.52s/it]


In [17]:
spectra_cells_s.T.applymap(lambda x: x["std_fit"] if isinstance(x, dict) \
                         else numpy.NaN).style.background_gradient(cmap ='YlOrRd',axis=None)

Scatterer_x,"(-25,)","(-20,)","(-15,)","(-10,)","(-5,)","(0,)","(5,)","(10,)","(15,)","(20,)"
Scatterer_y,,,,,,,,,,
-25,2.244640,1.234982,1.448526,1.299628,0.980051,1.106651,2.738870,2.632940,3.980819,6.322709
-20,0.982560,0.975286,0.942824,0.906108,0.950318,0.852315,0.959139,1.726270,2.117568,7.336354
-15,1.167148,0.866389,0.967204,0.900868,0.906533,0.863969,1.130486,1.599478,4.500799,12.106440
-10,2.104344,1.033364,0.953669,0.872521,0.823397,0.809743,1.315964,1.768697,4.656820,3.817858
-5,1.089080,1.047578,0.915401,0.882770,0.919733,1.087498,3.036959,4.688673,4.977495,11.756545
0,2.257738,3.051193,2.368236,3.232486,1.550989,4.994635,3.903326,3.312687,3.562746,1.655870
5,11.748825,3.015774,5.467810,3.758718,2.490061,11.890817,2.761383,4.159162,3.960486,1.123126
10,13.009521,4.699728,3.486851,7.913294,8.717503,11.703206,4.743916,4.414389,5.787814,0.154048
15,2.835980,10.470289,12.841282,13.223338,13.541500,3.840033,7.971243,8.236743,19.679759,0.258760


In [18]:
spectra_cells_s.T.applymap(lambda x: x["count"] if isinstance(x, dict) \
                         else numpy.NaN).style.background_gradient(cmap ='YlOrRd',axis=None)

Scatterer_x,"(-25,)","(-20,)","(-15,)","(-10,)","(-5,)","(0,)","(5,)","(10,)","(15,)","(20,)"
Scatterer_y,,,,,,,,,,
-25,96.000000,161.000000,129.000000,155.000000,150.000000,137.000000,32.000000,8.000000,9.000000,4.000000
-20,172.000000,942.000000,822.000000,709.000000,803.000000,566.000000,208.000000,84.000000,55.000000,7.000000
-15,147.000000,816.000000,859.000000,779.000000,809.000000,592.000000,207.000000,92.000000,32.000000,10.000000
-10,104.000000,569.000000,685.000000,525.000000,512.000000,371.000000,112.000000,39.000000,26.000000,5.000000
-5,75.000000,269.000000,293.000000,249.000000,206.000000,137.000000,71.000000,22.000000,22.000000,5.000000
0,20.000000,71.000000,74.000000,55.000000,57.000000,30.000000,30.000000,31.000000,27.000000,3.000000
5,7.000000,41.000000,47.000000,38.000000,27.000000,37.000000,23.000000,19.000000,11.000000,2.000000
10,6.000000,30.000000,31.000000,19.000000,13.000000,12.000000,21.000000,23.000000,15.000000,2.000000
15,5.000000,16.000000,17.000000,12.000000,14.000000,16.000000,19.000000,16.000000,10.000000,1.000000


In [19]:
spectra_cells_a.T.applymap(lambda x: x["std_fit"] if isinstance(x, dict) \
                         else numpy.NaN).style.background_gradient(cmap ='YlOrRd',axis=None)

Absorber_x,"(-25,)","(-20,)","(-15,)","(-10,)","(-5,)","(0,)","(5,)","(10,)","(15,)","(20,)"
Absorber_y,,,,,,,,,,
-25,2.459177,1.822108,1.795760,2.089643,1.671435,3.630047,2.886213,4.720930,8.503557,0.359445
-20,1.778155,0.857745,0.936532,0.961452,0.914372,0.982495,1.417018,2.864328,5.636495,10.045144
-15,1.591176,0.820477,0.887372,0.898306,0.880110,0.963012,1.052683,1.284687,3.144962,1.369870
-10,3.030947,0.946464,0.959786,0.991800,1.085293,1.174679,1.862059,3.538787,4.812424,5.868006
-5,2.118173,0.924056,0.817670,1.104257,1.108373,2.810130,2.544156,3.791865,9.370424,6.746083
0,10.567897,2.675828,0.948490,3.148790,2.720962,3.484773,6.346810,4.880619,3.152575,0.249624
5,8.268062,2.088171,1.929255,13.552963,3.895735,4.670742,6.205777,3.466824,17.317024,0.106283
10,6.167202,4.293339,0.766431,4.086676,3.859423,4.855694,3.220024,3.551209,7.343081,3.478462
15,10.223030,13.145636,3.780404,4.515519,16.120136,12.997471,13.558805,15.865348,6.117578,0.258761


In [20]:
spectra_cells_a.T.applymap(lambda x: x["count"] if isinstance(x, dict) \
                         else numpy.NaN).style.background_gradient(cmap ='YlOrRd',axis=None)

Absorber_x,"(-25,)","(-20,)","(-15,)","(-10,)","(-5,)","(0,)","(5,)","(10,)","(15,)","(20,)"
Absorber_y,,,,,,,,,,
-25,69.000000,89.000000,70.000000,89.000000,63.000000,59.000000,22.000000,10.000000,9.000000,2.000000
-20,133.000000,547.000000,543.000000,337.000000,300.000000,203.000000,92.000000,45.000000,23.000000,6.000000
-15,94.000000,635.000000,2850.000000,1668.000000,965.000000,435.000000,215.000000,152.000000,38.000000,2.000000
-10,65.000000,287.000000,896.000000,780.000000,480.000000,272.000000,98.000000,72.000000,22.000000,3.000000
-5,34.000000,145.000000,236.000000,231.000000,167.000000,108.000000,59.000000,40.000000,18.000000,4.000000
0,14.000000,67.000000,88.000000,69.000000,52.000000,53.000000,21.000000,29.000000,3.000000,1.000000
5,15.000000,49.000000,75.000000,37.000000,31.000000,23.000000,27.000000,29.000000,9.000000,2.000000
10,4.000000,42.000000,64.000000,35.000000,24.000000,26.000000,39.000000,34.000000,11.000000,3.000000
15,4.000000,9.000000,17.000000,12.000000,5.000000,8.000000,9.000000,8.000000,4.000000,2.000000


## Direction reconstruction

Setup:
- iTED A and C with the absorber and scatterer 

In [32]:
file_pos  = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_25_1.root"
file_pos1 = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_9_0.root"

f_root = ROOT.TFile.Open(file_pos)
t_root = f_root.COINCIDENCES

cut, cut_s = 0, 8

canvas = ROOT.TCanvas()
canvas.cd()

t_root.Draw(
    "deposited_energy[0]+deposited_energy[4]>>htemp(1400,100,1500)",
   f"deposited_energy[0]+deposited_energy[4]>0&\
    &detector_multiplicity==3&\
    &x[4]>-{25-cut}&x[4]<{25-cut}&\
    &y[4]>-{25-cut}&y[4]<{25-cut}&\
    &x[0]>-{25-cut_s}&x[0]<{25-cut_s}&\
    &y[0]>-{25-cut_s}&y[0]<{25-cut_s}",
    ""
)

h_root = ROOT.gPad.GetPrimitive("htemp")

h_root.Draw()
canvas.Draw()

In [38]:
file_pos  = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_25_1.root"
file_pos1 = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_9_0.root"

f_root = ROOT.TFile.Open(file_pos)
t_root = f_root.COINCIDENCES

cut, cut_s = 0, 0

canvas = ROOT.TCanvas()
canvas.cd()

t_root.Draw(
    "deposited_energy[0]+deposited_energy[4]>>htemp(1400,50,1500)",
   f"deposited_energy[0]+deposited_energy[4]>0&\
    &detector_multiplicity==2&\
    &x[4]>-{25-cut}&x[4]<{25-cut}&\
    &y[4]>-{25-cut}&y[4]<{25-cut}&\
    &x[0]>-{25-cut_s}&x[0]<{25-cut_s}&\
    &y[0]>-{25-cut_s}&y[0]<{25-cut_s}",
    ""
)

h_root = ROOT.gPad.GetPrimitive("htemp")

h_root.Draw()
canvas.Draw()

In [36]:
file_pos  = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_25_1.root"
file_pos1 = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_9_0.root"

f_root = ROOT.TFile.Open(file_pos)
t_root = f_root.COINCIDENCES

cut, cut_s = 0, 0

canvas = ROOT.TCanvas()
canvas.cd()

t_root.Draw(
    "Delta_t[0]-Delta_t[4]",
   f"deposited_energy[0]+deposited_energy[4]>0&\
    &detector_multiplicity==3&\
    &Delta_t[0]-Delta_t[4]!=0&\
    &x[4]>-{25-cut}&x[4]<{25-cut}&\
    &y[4]>-{25-cut}&y[4]<{25-cut}&\
    &x[0]>-{25-cut_s}&x[0]<{25-cut_s}&\
    &y[0]>-{25-cut_s}&y[0]<{25-cut_s}",
    ""
)

h_root = ROOT.gPad.GetPrimitive("htemp")

h_root.Draw()
canvas.Draw()

In [24]:
file_pos  = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_25_1.root"
file_pos1 = "/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTED_ACFar_BD_Near_Cs137_iTEDABack_D.2023_05_03_T.18_01_00_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100_9_0.root"

f_root = ROOT.TFile.Open(file_pos)
t_root = f_root.COINCIDENCES

cut, cut_s = 0, 0

canvas = ROOT.TCanvas()
canvas.cd()

t_root.Draw(
    "Delta_t[0]-Delta_t[4]",
   f"deposited_energy[0]>240.0&\
    &deposited_energy[4]>150&\
    &x[4]>-{25-cut}&x[4]<{25-cut}&\
    &y[4]>-{25-cut}&y[4]<{25-cut}&\
    &x[0]>-{25-cut_s}&x[0]<{25-cut_s}&\
    &y[0]>-{25-cut_s}&y[0]<{25-cut_s}",
    ""
)

h_root = ROOT.gPad.GetPrimitive("htemp")

h_root.Draw()
canvas.Draw()

## Conclusions

Some notes:
- Although not noticeable it's worth taking into account the limitation of not being able to calculate very small numbers with c++'s pow
- Using the energy-weighted results in smaller tails even if for comparable standard deviations (plotted below)

Improvements:
- Analyse all the absorbers to have a more complete picture of the timing between planes
- Measurement with source in different positions to have statistics in all parts of the crystal

In [25]:
spectra[0][9]["histo"].Draw()
spectra[1][25]["histo"].Draw()

In [26]:
spectra_cuts[0][0]["histo"].Draw()
spectra_cuts[8][0]["histo"].Draw()

## All plots

In [27]:
for i_skew in skew:
    for i_degree in degree:        
        spectra[i_degree][i_skew]["histo"].Draw()